In [6]:
import bqplot
import os
from bqplot_image_gl import ImageGL
from astropy.io import fits
from astropy.visualization import PercentileInterval
from bqplot import Figure, LinearScale, Axis, ColorScale
from bqplot import pyplot as plt
from bqplot import Toolbar
import numpy as np
import ipyvuetify as v
from ipywidgets import Layout
import ipywidgets as widgets
import traitlets
from astroquery.mast import Observations
import requests
from io import BytesIO

In [7]:
# this is a widget for debugging
out = widgets.Output()
out.add_traits(_metadata=traitlets.Dict(default_value={'mount_id': 'out'}).tag(sync=True))
out

Output()

In [8]:
# set the STIS targets for demo 
targets = ['NGC346', 'SN2001EP']
obsid = ['odg602060', 'o6gl37030']
tozip = list(zip(targets, obsid))

In [9]:
def get_xy(data):
    ''' get the xy data '''
    wave = data['WAVELENGTH']
    flux = data['FLUX']
    return wave, flux

In [10]:
def create_spec1d(x, y):
    ''' create a bqplot line chart for a 1d spectrum '''
    fig = plt.figure(title='Spectrum 1d')
    p = plt.plot(x, y)
    fig.layout.width = 'auto'
    fig.layout.height = 'auto'
    fig.layout.min_height = '400px' # so it shows nicely in the notebook
    return fig, p

In [11]:
def create_spec2d_heatmap(data):
    ''' create a 2d spectrum view as a bqplot Heatmap'''
    # trim the data to 95% of values
    i = PercentileInterval(95)
    limdata = i(data)

    # create the heatmap
    spec2d = plt.figure(padding_y=0)
    plt.scales(scales={'color': ColorScale(scheme='Greys', reverse=True)})
    axes_options = {'color': {'visible': False}}
    heat = plt.heatmap(limdata, axes_options=axes_options)
    return spec2d, heat

In [12]:
def create_cutout(data, target=None):
    ''' create image cutout as bqplot Heatmap '''
    aspect_ratio = data.shape[1] / data.shape[0]
    img = plt.figure(title='Src {0}'.format(target), layout=Layout(width='500px', height='500px'),
                     min_aspect_ratio=aspect_ratio, 
                     max_aspect_ratio=aspect_ratio, padding_y=0)
    plt.scales(scales={'color': ColorScale(scheme='Greys', reverse=True)})
    axes_options = {'x': {'visible': False}, 'y': {'visible': False}, 'color': {'visible': False}}
    p = plt.heatmap(data, axes_options=axes_options)
    return img, p

In [13]:
def retrieve_obs(targetid):
    ''' retrieve observation of target using astroquery '''
    try:
        ot = Observations.query_criteria(objectname=targetid, project='HST', radius='0.5 arcmin', intentType='science')
    except Exception:
        return None
    else:
        dp = Observations.get_product_list(ot)
        return dp
    
def get_type(dp, datatype, groupby=None):
    ''' filter data product list by image/spectrum types '''
    sub = ['RAW'] if datatype == 'image' else ['X1D', 'X2D']
    fp = Observations.filter_products(dp, productSubGroupDescription=sub, dataproduct_type=datatype)
    g = fp.group_by('obs_id')
    if groupby:
        g = g.groups[g.groups.keys['obs_id']==groupby]
    return g

def get_uri(table):
    ''' extract the data URI from the data products table '''
    return table['dataURI'].tolist()

def get_remote_hdu(uri):
    ''' get a remote HDU of an observation '''
    url = 'https://mast.stsci.edu/api/v0.1/Download/file?uri={0}'.format(uri)
    r = requests.get(url)
    hdu = fits.open(BytesIO(r.content))
    return hdu

def get_img_cut(im, size=50):
    ''' cut out portion of image data for demo purposes '''
    xcen, ycen = int(im.header['CRPIX1']), int(im.header['CRPIX2'])
    return im.data[xcen-size:xcen+size,ycen-size:ycen+size]

def get_observation(target, obs):
    ''' get image cutout, 1d and 2d spectrum '''
    dp=retrieve_obs(target)
    if not dp:
        raise RuntimeError('No observations found for {0}'.format(target))

    cut = get_remote_hdu(get_uri(get_type(dp, 'image'))[0])
    cutimg = get_img_cut(cut[1], size=100)
    x1d, x2d = [get_remote_hdu(u) for u in get_uri(get_type(dp, 'spectrum', groupby=obs))]
    return cutimg, x1d, x2d

In [14]:
@out.capture()
def update_data(selected_src):

    # get the target STIS observation
    targ, obs = selected_src
    cut, hdu1d, hdu2d = get_observation(targ, obs)
    print('target', targ, obs, cut.shape)
    
    # update spec1d data
    wave, flux = get_xy(hdu1d[1].data)
    fig, plot = create_spec1d(wave, flux)

    spec2d, heat = create_spec2d_heatmap(hdu2d[1].data)
    
    img, image = create_cutout(cut, target=targ)
 
    children = [
            # load the histogram and slider content
            v.Col(xs12=True, lg6=True, xl4=True, children=[
                img
            ]),

            # load the line plot content
            v.Col(xs12=True, xl4=True, children=[
                spec2d
            ]),
            # load the line plot content
            v.Col(xs12=True, xl4=True, children=[
                fig
            ]),       
    ]
    row.children = children

In [15]:
@out.capture()
def update_data_handler(widget, event, data):
    # get new data from selected target
    print('targets', targets, data)
    selected_src = tozip[targets.index(data)]
    print('selected_src', selected_src)
    update_data(selected_src)

In [16]:
row = v.Row(_metadata={'mount_id': 'protospec'}, dense=True, row=True, wrap=True, align_center=True, children=['Loading..'])
row.on_event('change', update_data_handler)

In [17]:
# create an app bar to load a select button
btn1 = v.Select(dense=True, outlined=True, label='Target', items=targets)
btn1.on_event('change', update_data_handler)
v.AppBar(_metadata={'mount_id': 'appbar'}, app=True, absolute=True, children=[
    v.Col(class_='col-md-2 col-lg-2', children=[btn1])
])

AppBar(absolute=True, app=True, children=[Col(children=[Select(dense=True, items=['NGC346', 'SN2001EP'], label…